# 3. Analysis by contaminant
In this section, we analyse air pollution by contaminants. 

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import plotly.offline as pyo
import plotly.graph_objs as go
from plotly import tools
import plotly.figure_factory as ff

In [18]:
# display the plotly plots inside the Jupyter notebook
pyo.init_notebook_mode(connected=True)

In [9]:
# load data
path = "/Users/linetonthat/ds_training/projects/2019-06_Harvey_pollution/3_treated_data/"
df = pd.read_csv(path+"largest-emissions-in-lbs.csv")
df.head()

,report_id,Event began,Event ended,Regulated entity RN number,Regulated entity name,Type(s) of air emissions event,County,contaminant,authorization,limit,quantity,units
0,266261,2017-08-27 00:00:00,2017-09-06 00:00:00,RN103919817,CHEVRON PHILLIPS CHEMICAL CEDAR BAYOU PLANT,AIR SHUTDOWN,HARRIS,Carbon Monoxide,1504A,1892.04 LBS/HR,244040.0,lbs (est.)
1,266378,2017-08-29 11:50:00,2017-09-01 11:50:00,RN100217389,FLINT HILLS RESOURCES PORT ARTHUR FACILITY,AIR SHUTDOWN,JEFFERSON,Carbon Monoxide,No specific Authorization,0.0,240000.0,lbs (est.)
2,267078,2017-09-08 08:00:00,2017-09-22 08:00:00,RN100217389,FLINT HILLS RESOURCES PORT ARTHUR FACILITY,AIR STARTUP,JEFFERSON,Ethylene (gaseous),No specific authorization,0.0,150000.0,lbs (est.)
3,267078,2017-09-08 08:00:00,2017-09-22 08:00:00,RN100217389,FLINT HILLS RESOURCES PORT ARTHUR FACILITY,AIR STARTUP,JEFFERSON,Carbon Monoxide,No specific authorization,0.0,150000.0,lbs (est.)
4,266566,2017-09-01 05:00:00,2017-10-01 05:00:00,RN100221662,EQUISTAR CORPUS CHRISTI PLANT,AIR STARTUP,NUECES,Carbon Monoxide,Permit 83864,721.67 LBS/HR,121000.0,lbs (est.)


## Revised and categorised list of contaminants
As the designation of contaminants was not consistent in the dataset, I've reviewed it to provide a more consistent designation when relevant, as well as categories of pollutants. Not being an expert in air pollution by petrochemical industry, I've asked a friend who is to help me with this task!

-> The result is available in the "list_of_contaminants_v2.csv" file.

In [10]:
# load categorised list of contaminants
df_conta = pd.read_csv(path+"../4_analyses/contaminants/list_of_contaminants_v2.csv", sep = ";")
drop_question = lambda x: x.replace("?",'')
df_conta['category'] = df_conta['category'].apply(drop_question)
df_conta.head()

,ID,contaminant,consistent_designation,carbon no (for hydrocarbon),category
0,25,Butane,Butanes,C4,Hydrocarbons
1,26,"Butane, N-",Butanes,C4,Hydrocarbons
2,27,"Butane, i",Butanes,C4,Hydrocarbons
3,28,Butanes,Butanes,C4,Hydrocarbons
4,156,n-butane,Butanes,C4,Hydrocarbons


In [11]:
# add the category column to the main dataframe
df = pd.merge(df,df_conta[['contaminant','category']], on = ['contaminant'])
df.tail()

,report_id,Event began,Event ended,Regulated entity RN number,Regulated entity name,Type(s) of air emissions event,County,contaminant,authorization,limit,quantity,units,category
1450,266269,2017-08-27 10:45:00,2017-09-07 10:45:00,RN100224815,PASADENA TERMINAL,EMISSIONS EVENT,HARRIS,t-octene-2,NSR 5171,0.0,0.9473,lbs (est.),Hydrocarbons
1451,266556,2017-08-30 08:00:00,2017-08-31 20:00:00,RN100224815,PASADENA TERMINAL,EMISSIONS EVENT,HARRIS,t-octene-2,NSR 5171,0.0,0.1510,lbs (est.),Hydrocarbons
1452,266269,2017-08-27 10:45:00,2017-09-07 10:45:00,RN100224815,PASADENA TERMINAL,EMISSIONS EVENT,HARRIS,2-methylnaphthalene,NSR 5171,0.0,0.2932,lbs (est.),Hydrocarbons
1453,266556,2017-08-30 08:00:00,2017-08-31 20:00:00,RN100224815,PASADENA TERMINAL,EMISSIONS EVENT,HARRIS,2-methylnaphthalene,NSR 5171,0.0,0.0467,lbs (est.),Hydrocarbons
1454,266116,2017-09-04 07:49:00,2017-09-04 12:21:00,RN100218973,FORMOSA POINT COMFORT PLANT,AIR STARTUP,CALHOUN,Ethyl mercaptan,19168/PSDTX1226,761.65 LBS/HR,0.0200,lbs (est.),Organosulfur compounds


In [12]:
df['category'].nunique()

20

In [13]:
df['category'].unique()

array(['Carbon Monoxide', 'Hydrocarbons', 'Sulfur dioxide', 'VOCs', 'TBD',
       'Particulate Matter', 'NOx', 'Methane', 'solvant', 'Mineral Spirit',
       'Hydrogen', 'Hydrogen Sulfide', 'Ammonia', 'Chlorofluorocarbons',
       'Organosulfur compounds', 'Hydrogen cyanide', 'Opacity',
       'Other material', 'Heavy metals', 'Carbon Dioxide'], dtype=object)

## Overall contaminant emissions

In [15]:
# assess quantities per contaminant type
df2 = df.groupby(by=['category'])[['quantity']].sum()
df2.columns = ['quantity']
df2.sort_values(by = ['quantity'], ascending = False, inplace = True)
print(df2)

                            quantity
category                            
Carbon Monoxide         1.993716e+06
Hydrocarbons            1.611702e+06
Sulfur dioxide          6.365035e+05
NOx                     4.226846e+05
VOCs                    2.768571e+05
TBD                     2.512821e+05
Methane                 9.600887e+04
Particulate Matter      8.626717e+04
solvant                 1.682786e+04
Mineral Spirit          1.117539e+04
Hydrogen Sulfide        9.303650e+03
Hydrogen                5.130230e+03
Ammonia                 1.716100e+03
Chlorofluorocarbons     4.000000e+02
Organosulfur compounds  2.000200e+02
Hydrogen cyanide        1.130000e+02
Opacity                 8.480000e+00
Other material          4.000000e+00
Heavy metals            1.203000e+00
Carbon Dioxide          1.010000e+00


In [22]:
# draw horizontal bar chart of main pollutants emitted
overall_title = "Harvey-related emission from the chemical plants"
x_title = "Quantities (lbs)"
### --------------------------------------------------------

trace= [go.Bar(
        y=df2.index,
        x=df2['quantity'],
        orientation = 'h'
        )]
layout = go.Layout(title = overall_title, 
                   hovermode = 'closest',
                   xaxis=dict(dict(title=x_title,
                                   domain=[0.25, 1.0], anchor='y1')),
                   yaxis=dict(dict(domain=[0.0, 1.0], anchor='x1')))
fig = go.Figure(data = trace, layout = layout)
pyo.iplot(fig)

Carbon monoxide is the most emitted contaminant with approximately 2 million pounds emitted, followed by hydrocarbons with 1.6 million pounds emitted, and sulfur dioxide with 0.6 million pounds emitted.

<color = orange> XX  comparison with normal levels</colour>
## Contaminant emissions by facility

In [23]:
# assess quantities per contaminant type and per facility
df3 = pd.pivot_table(df, values = 'quantity', index = 'category', columns = 'Regulated entity name',
                     aggfunc = 'sum')

In [24]:
# draw horizontal bar chart of pollutants emitted by facility
# select the facility
facility = 'ARKEMA CROSBY PLANT' #df3.columns[0]
### --------------------------------------------------------
overall_title = "Contaminants - "+ facility
x_title = "Quantities (lbs)"
### --------------------------------------------------------
# select only emitted contaminants to be displayed
b_df = df3[[facility]].dropna().sort_values(by = [facility], ascending = False)
### --------------------------------------------------------
trace= [go.Bar(
        y=b_df.index,
        x=b_df[facility],
        orientation = 'h'
        )]
layout = go.Layout(title = overall_title, 
                   hovermode = 'closest',
                   xaxis=dict(dict(title=x_title,
                                   domain=[0.25, 1.0], anchor='y1')),
                   yaxis=dict(dict(domain=[0.0, 1.0], anchor='x1')))
fig = go.Figure(data = trace, layout = layout)
pyo.iplot(fig)

## Contaminant emissions by county

In [25]:
# assess quantities per contaminant type and per county
df4 = pd.pivot_table(df, values = 'quantity', index = 'category', columns = 'County',
                     aggfunc = 'sum')

In [26]:
# draw horizontal bar chart of pollutants emitted by county
# select the county
county = 'BRAZORIA' #df4.columns[0]
### --------------------------------------------------------
overall_title = "Contaminants - " + county
x_title = "Quantities (lbs)"
### --------------------------------------------------------
# select only emitted contaminants to be displayed
b_df = df4[[county]].dropna().sort_values(by = [county], ascending = False)
### --------------------------------------------------------
trace= [go.Bar(
        y=b_df.index,
        x=b_df[county],
        orientation = 'h'
        )]
layout = go.Layout(title = overall_title, 
                   hovermode = 'closest',
                   xaxis=dict(dict(title=x_title,
                                   domain=[0.25, 1.0], anchor='y1')),
                   yaxis=dict(dict(domain=[0.0, 1.0], anchor='x1')))
fig = go.Figure(data = trace, layout = layout)
pyo.iplot(fig)